In [23]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
import plotly
import math
from statistics import median, mean
from selfpkg import readDB, MinToMS
from dataFilter import butter_lowpass_filter, butter_highpass_filter

In [24]:
timeConversionVal = 60000

In [25]:
#function for calculate the time interval between peaks
def velocityCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = dfTime[i] - prev #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [26]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [27]:
def jointDetectionByRotationalSpeed(df, filename='jointDetectionByRotationalSpeed', freq=1):
    x = df.rotationalSpeedX
    y = df.rotationalSpeedY
#     z = df.rotationalSpeedZ

#     v = (x ** 2 + y ** 2) ** (1/2)
    v = y

    v_processed = v

    for i in range(len(v_processed)): #eliminate peak smaller than 100
        if v_processed[i] < 600:
            v_processed[i] = 0

#     i = 0
#     j = 0
#     width = 25 #ger rid of peak whose width > 25 points
#     while(j < len(v_processed)):
#         if v_processed[i] == 0 and v_processed[j] == 0:
#             i = i + 1
#             j = j + 1
#         elif v_processed[i] != 0 and v_processed[j] != 0:
#             j = j + 1
#         elif v_processed[i] != 0 and v_processed[j] == 0:
#             if j - i + 1 > width:
#                 for k in range(i, j):
#                     v_processed[k] = 0
#             i = j;

    #generate peaks index
    peaksIndex, _ = ss.find_peaks(v_processed)
    #merge close peaks
    i = 0
    j = 1
    thresInterval = 70 # the smallest interval between two joints
    res = []
    while(j < len(peaksIndex)):
        if  j == len(peaksIndex) - 1:
            if v_processed[peaksIndex[i]] >= v_processed[peaksIndex[j]]:
                res.append(peaksIndex[i])
            else:
                res.append(peaksIndex[j])
            i = j
            j += 1       
        else:
            if peaksIndex[j] - peaksIndex[j-1] > thresInterval:
                j -= 1
                if v_processed[peaksIndex[i]] >= v_processed[peaksIndex[j]]:
                    res.append(peaksIndex[i])
                else:
                    res.append(peaksIndex[j])
                j += 1
                i = j
                j += 1       
            else:
                j += 1

    res = res[1:]

    trace0 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=v,
        name = 'rotational speed',
        line = dict(
            color = ('rgb(25, 25, 25)'),
            width = 1)
    )

    trace1 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=v_processed,
        name = 'processed rotational speed',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )

    trace2 = go.Scattergl(
        x = df.time[res] / timeConversionVal,
        y = v_processed[res],
        mode = 'markers',
        marker = dict(
            color = ('rgb(225, 0, 0)'),
            size = 5
        )
    )

    trace3 = go.Scattergl(
        x = df.time[peaksIndex] / timeConversionVal,
        y = v_processed[peaksIndex],
        mode = 'markers',
        marker = dict(
            color = ('rgb(0, 0, 225)'),
            size = 5
        )
    )
    
    traceJoint = go.Scattergl(
        x = df.time[res] / timeConversionVal,
        y = [-1] * len(res),
        mode = 'markers',
        name = 'joint',
        marker = dict(
            color = ('rgb(225, 0, 0)'),
            size = 5
        )
    )
    
    traceCompass = go.Scattergl(
        x = df.time / timeConversionVal,
        y = (df.compassX ** 2 + df.compassY ** 2 + df.compassZ ** 2 - 3500) / 100,
        name = 'compass',
        mode = 'lines',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1
        )
    )
    
    trace00 = go.Scattergl(
        x=df.time / timeConversionVal, 
        y=df.flapper0, #butter_lowpass_filter(df.flapper0, freq, 50, 5), #
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace01 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper1, #butter_lowpass_filter(df.flapper1, freq, 50, 5), #
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace02 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper2, #butter_lowpass_filter(df.flapper2, freq, 50, 5), #
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace03 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper3, #butter_lowpass_filter(df.flapper3, freq, 50, 5),  #
        name = 'flapper3',
        line = dict(
            color = ('rgb(0, 220, 255)'),
            width = 1)
    )
    trace04 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper4, #butter_lowpass_filter(df.flapper4, freq, 50, 5), #
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace05 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper5, #butter_lowpass_filter(df.flapper5, freq, 50, 5),  #
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace06 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper6, #butter_lowpass_filter(df.flapper6, freq, 50, 5), #
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace07 = go.Scattergl(
        x=df.time / timeConversionVal,
        y=df.flapper7, #butter_lowpass_filter(df.flapper7, freq, 50, 5), #
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    data = [trace1, trace3, trace2, traceJoint, traceCompass, trace00, trace01, trace02, trace03, trace04, trace05, trace06, trace07]#, traceAcc]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Rotational Speed(rad/s)')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')
    
    return res

In [28]:
def speedCalculation(df, res, filename='speedCalculation'):
    time = velocityCal(df.time, res)

    #calculate mean of velocity
    errorPercent = 0.2
    mode = 6000

    con = []
    avg = []

    time = time.reset_index(drop=True)
    window = 5;

    avg = movingaverage(time, 10)
    
#     avgAll = mean(time)
#     for i in time.index:
#         if time[i] > 2 * avgAll or time[i] < 1/2 * avgAll:
#             del time[i]
    
    
    for i in time.index:    
#         try:
#             # shift those points with double interval to avg, optional
#             if i > 0 and i < len(time.index) and \
#                 time[i] > (2 - errorPercent) * time[i-1] and \
#                 time[i] > (2 - errorPercent) * time[i+1]:
#                 time[i] = time[i] #optional
# #                 time[i] = time[i-1]
#         except KeyError:
#             time[i] = time[i]
#         finally:   
            diff = abs(time[i] - avg[i])
            con.append(1 - diff/avg[i])

    #fill the result to df
    df['velocity'] = 0
    df['probability'] = 0
    df['avg'] = 0
    df['velocity'][res] = time
    df['probability'][res] = con
    df['avg'][res] = avg

    shrinkVal = 0.1 #other points between two joint, lower the possibility
    thresPro = 0.6 #threshold on probability for selecting a joint

    first = []
    #fill out empty velocity & probability in df
    #connect tow close confident points(probability > thresPro), calculate points between them by linear function
    #first & last segment, only use one point
    for index, row in df.iterrows():
        if len(first) == 0:
            if row['probability'] < thresPro:
#             if row['velocity'] > 1.5 * row['avg']:
                continue
            else:
                first = [row['velocity'], row['probability'], row['time'], index]
                for i in range(0, index):
                    df['velocity'][i] = first[0]
                    df['probability'][i] = first[1] - shrinkVal
        else:
            if index != len(df.index) -1:
                if row['probability'] < thresPro:
#                 if row['velocity'] > 1.5 * row['avg']:
                    continue
                else:
                    k = (row['velocity'] - first[0]) / (row['time'] - first[2])
                    b = row['velocity'] - k * row['time']
                    for i in range(first[3]+1, index):
                        avg2 = (df['avg'][index] + df['avg'][first[3]]) / 2
                        df['velocity'][i] = k * df['time'][i] + b
                        df['probability'][i] = (1 - abs(df['velocity'][i] - avg2) / avg2) - shrinkVal
                    first = [row['velocity'], row['probability'], row['time'], index]
            else:
                for i in range(first[3], len(df.index)):
                    df['velocity'][i] = first[0]
                    df['probability'][i] = first[1] - shrinkVal
    
#     ratio = 0.05
#     mvAvg = velocityFromAcc(df)
#     meanFront = abs(mean(mvAvg[0: int(ratio * len(df.index))]))
#     meanBack = abs(mean(mvAvg[int((1-ratio) * len(df.index)): len(df.index)]))
#     for index, row in df.iterrows():
#         if row['time'] < df.time.tail(1).tolist()[0] * ratio:
#             if abs(mvAvg[index]) < 15 and meanFront < 15:
#                 df['velocity'][index] = df['velocity'][int(ratio * len(df.index))]
#         if row['time'] > df.time.tail(1).tolist()[0] * (1-ratio):
#             if abs(mvAvg[index]) < 15 and meanBack < 15:
#                 df['velocity'][index] = df['velocity'][int((1-ratio) * len(df.index))]
            
    timeConversionVal = 60000

    traceAcceleration = go.Scattergl(
        x = df.time[res]/timeConversionVal,
        y = time,
        mode = 'markers',
        name = 'compass',
        marker = dict(
            color = ('rgb(0, 0, 255)')
        )
    )

    traceFit = go.Scattergl(
        x = df.time / timeConversionVal,
        y = df.velocity,
        mode = 'lines',
        name = 'velocity',
        line = dict(
            color = ('rgb(127, 127, 127)')
        )
    )


    data = [traceAcceleration, traceFit]

    layout = dict(title = filename,
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'timeInterval')
                  )
    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') 

In [29]:
def speedPlot(df, filename='speedPlot'):
    
    lenOfPipe = 40
    
    df['velocity'] = lenOfPipe / (df['velocity']/1000) #trans from time interval to velocity
    df['avg'] = lenOfPipe / (df['avg']/1000)
    
    traceV1 = go.Scattergl(
        x = df.time / timeConversionVal,
        y = df.velocity,
        mode = 'lines',
        name = 'Velocity',
        line = dict(
            color = ('rgb(0, 0, 255)')
        )
    )
    
    traceP1 = go.Scattergl(
        x = df.time / timeConversionVal,
        y = df.probability,
        mode = 'lines',
        name = 'Probability',
        line = dict(
            color = ('rgb(255, 0, 0)')
        )
    )
    
    data = [traceV1, traceP1]
    
    layout = dict(title = filename,
                      xaxis = dict(title = 'Time(minutes)'),
                      yaxis = dict(title = 'speed(feet/sec)'))
    
    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') 

In [30]:
run = 6
df = pd.read_csv('/Users/meng/Downloads/Wise_Run' + str(run) + '_Trimed.csv')
res = jointDetectionByRotationalSpeed(df, 'jointDetection' + str(run), freq=0.5)

In [31]:
speedCalculation(df, res, 'speedCalculation' + str(run))
speedPlot(df, 'speedPlot' + str(run))

In [51]:
df.to_csv('wise' + str(run) + ' with speed.csv')

In [52]:
df = readDB('sqlite:///wise06.db')

In [121]:
def testChart(df, start=None, end=None, filename='testChart'):
         
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
        
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedX[startIndex: endIndex] / 10,
        name = 'rotationalSpeedX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedY[startIndex: endIndex] / 10,
        name = 'rotationalSpeedY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedZ[startIndex: endIndex] / 10,
        name = 'rotationalSpeedZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = ((df.rotationalSpeedX / 10) ** 2 + (df.rotationalSpeedY / 10) ** 2 + (df.rotationalSpeedZ / 10) ** 2) ** (1/2),
        name = 'rotationalSpeed',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1
        )
    )
    
    #compass
    trace4 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = (df.compassX ** 2 + df.compassY ** 2 + df.compassZ ** 2) ** (1/2) * 10 - 100,
        name = 'compass',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1
        )
    )

    data = [trace0, trace1, trace2, trace3, trace4]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Rotation Speed'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [1]:
run = 4
df = pd.read_csv('/Users/meng/Downloads/Wise_Run' + str(run) + '_Trimed.csv')
testChart(df)

NameError: name 'pd' is not defined

In [20]:
#function for generating compass chart
def rotationalSpeedCompassChart(df, start=None, end=None, filename='rotationalSpeedCompassChart'):
         
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
        
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedX[startIndex: endIndex] / 50,
        name = 'rotationalSpeedX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedY[startIndex: endIndex] / 50,
        name = 'rotationalSpeedY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedZ[startIndex: endIndex] / 50,
        name = 'rotationalSpeedZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = (df.compassX ** 2 + df.compassY ** 2 + df.compassZ ** 2) ** (1/2) + 50,
        name = 'compass',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1
        )
    )

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Rotation Speed'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [21]:
run = 4
df = pd.read_csv('/Users/meng/Downloads/Wise_Run' + str(run) + '_Trimed.csv')

In [22]:
rotationalSpeedCompassChart(df)